In [1]:
import numpy as np
import pandas as pd

import sys
import re


#Это просто мой способ коннекта к LLM, тут может быть какой-то ваш коннектор
sys.path.append('../llms_local')
import chat_bot

E:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"



device cuda


In [2]:
#Это просто мой способ коннекта к LLM, тут может быть какой-то ваш коннектор
user_id = 3
creator_bot = chat_bot.ChatBot(user_id)
creator_bot.name_in_promt = "2-342043824308240834208"

In [3]:
class LLM_optimizer:
    def __init__(self, lang='only in English', eval_len=130, imporvement_len=200, eval_temp=0.1, improve_temp=1.1):
        self.lang = lang
        self.imporvement_len = imporvement_len
        self.eval_len = eval_len
        self.eval_temp = eval_temp
        self.improve_temp = improve_temp
        self.result = {}   
    def evaluate_response(self, response, success_criteria, initial_task, n_tokens=100):
        feedback = []
        total_quality_score = 0
        num_criteria = len(success_criteria)

        for criterion in success_criteria:
            evaluation_prompt = f"Есть задача: {initial_task}\n\n Есть ответ:{response}\n\n Оцени следующий ответ по критерию: '{criterion}'. Какую оценку ты поставил бы от 0 до 100 и почему? Оценка ({self.lang}):"
            evaluation_result = creator_bot.create_text(evaluation_prompt, False, max_tokens=n_tokens, temp=self.eval_temp)

            # Предполагаем, что результат содержит и оценку, и пояснение
            # Разделим результат на оценку и объяснение

            # Используем регулярное выражение для поиска оценки
            score_match = re.search(r'(\d+(\.\d+)?)', evaluation_result)  # Ищем число с возможной десятичной точкой

            if score_match:
                score = float(score_match.group(0))  # Получаем найденное число
            else:
                score = 0.0  # По умолчанию, если нет совпадений, устанавливаем оценку в 0

            # Извлекаем критику, оставив оставшуюся часть строки
            critique = evaluation_result
            total_quality_score += score

            feedback.append({
                'criterion': criterion,
                'score': score,
                'critique': critique.strip()
            })

        # Рассчитываем среднюю оценку
        average_quality_score = total_quality_score / num_criteria if num_criteria > 0 else 0

        return {'quality': average_quality_score, 'feedback': feedback}

    def improve_response(self, response, feedback, initial_task, n_tokens=220, temp=0.8):
        # Объединяем критику и создаем промт для улучшения
        critique_notes = "\n".join([f"Критика по критерию '{f['criterion']}': {f['critique']}" for f in feedback])

        improvement_prompt = f"Есть задача: {initial_task}\n\n Есть ответ:{response}\n\n Напиши лучший ответ с учетом этой критики (не пиши свою оценку ответа, только улучшенный ответ!):\n{critique_notes}\nРазмер поля ввода {int(n_tokens/3)} слов\nОтвет ({self.lang}):"
        improved_response = creator_bot.create_text(improvement_prompt, False, max_tokens=n_tokens, temp=temp)
        improved_response = creator_bot.filtration(improved_response)
        return improved_response

    def iterative_improvement(self, initial_task, success_criteria, max_iterations=10, desired_quality=8.0, temp=0.8):
        # Создание начального ответа
        initial_prompt = f"Сформулируй начальный ответ на следующую задачу: {initial_task}. Ответ ({self.lang}):"
        current_response = creator_bot.create_text(initial_prompt, False, max_tokens=self.imporvement_len, temp=0.1)
        current_response = creator_bot.filtration(current_response)

        iteration_count = 0

        best_result = {'quality':0, 'value':'', 'feedback':''}
        while iteration_count < max_iterations:
            # Оценка текущего ответа на основе критериев успеха
            feedback = self.evaluate_response(current_response, success_criteria, initial_task, n_tokens=self.eval_len)

            print("****current_response", current_response)
            print("-----feedback['feedback']", feedback['feedback'])
            # Проверка, достигнуто ли желаемое качество
            if feedback['quality'] >= desired_quality:
                print(f"Достигнуто желаемое качество: {feedback['quality']}.")
                break
            if feedback['quality'] > best_result['quality']:
                best_result['value'] = current_response
                best_result['quality'] = feedback['quality']
                best_result['feedback'] = feedback['feedback']
                print('UPDATE', feedback['quality'], best_result['quality'])
                self.result = best_result
            else:
                current_response = best_result['value']
                feedback['feedback'] = best_result['feedback']
                print('REJECT', feedback['quality'], best_result['quality'])


            # Улучшение ответа на основе полученной обратной связи
            current_response = self.improve_response(current_response, feedback['feedback'], initial_task, self.imporvement_len, self.improve_temp)
            iteration_count += 1

        return best_result['value']  # Возвращение наиболее качественного ответа



In [4]:
opt = LLM_optimizer(eval_len=130, imporvement_len=350, eval_temp=0.1, improve_temp=1.5)

In [ ]:
# Пример использования функции
initial_task = "Teach me how to treat an open fracture in field"#
success_criteria = [
    "Specifics (0 - blurry, 100 - precise plan, understandable to a schoolchild or robot)",
    "Probability of success of this plan",
    "Ease of reading by a person who is not a doctor, and in front of whom a wounded man is now lying, in the field",
    "Efficiency (0 - will not improve the patient’s condition, 100 - will cure to the condition that was before the fracture)"
]


initial_task = "I want an improved version of LLM. I would like my LLM to be comparable in problem solving quality to GPT-4, but to be more lightweight and trained on less data in less time. How to do this? You can think as broadly as possible: the product does not have to be just a neuron, it also does not have to be a transformer."#
success_criteria = [
    "Specifics (0 - blurry, 100 - precise plan, understandable to a schoolchild or robot)",
    "Probability of success of this plan",
    "Simplicity (0 - requires 5 man-years, 100 - can be completed in one person in a few evenings)",
    "Efficiency (0 - will give a slight improvement in sample-efficiency, 100 - will reduce the need for data by orders of magnitude without loss of quality)"
]

initial_task = "I have 1 million rubles, I live in a house in Siberia, I need to make an aircraft of the MiG-17 level, with similar speed and range. Write out a step-by-step plan. Consider any possibilities - you are very purposeful, and are ready to take any workaround."#
success_criteria = [
    "Specifics (0 - minimal details, 100 - precise plan, understandable to a schoolchild or robot, with times, materials and company names)",
    "Probability of success of this plan",
    "Simplicity (0 - requires 5 man-years, 100 - can be completed in one person in a few evenings)"
]

initial_task = "Tell me what the fight between Pennywise and the Losers (2nd movie) would look like if they were all really smart and strived to achieve victory using the intelligence and resources of at least an average adult. For example, a smart Pennywise could subdue the police or bomb the car of the main characters. And smarter protagonists could have poured a tank of gasoline into his well. But this is just an example, come up with more options."
success_criteria = [
    "Specifics (0 - minimal details, 100 - precise plan, with times, items, weapons, organizations and etc)",
    "Plausibility (0 - a lot of God from the Machine, 100 - everything is very life-like and everyday)",
    "Characters' intelligence (0 - characters simply follow the plot, 100 - characters want to win quickly and return to their business, and for this they are ready to do any dirt)",
    "Intelligence of the antagonist (0 - Pennywise is a hungry stupid monster, 100 - Pennywise is a military genius and silent assassin)",
]

initial_task = "What is the optimal strategy and instrument for stock speculation?"
success_criteria = [
    "Specifics (0 - minimal details, 100 - precise plan, with time, items, instruments, algoritms, math and etc)",
    "Probability of success of this plan (0 - I will be in profit with 50% probability, 100 - I will be in profit with 100% probability)",
    "Usability (0 - I need many trials to succed in this, 100 - I can make money by this strategy right now)",
    "Reasonableness"
]


final_response = opt.iterative_improvement(initial_task, success_criteria, desired_quality=90.0, max_iterations=80)
print(final_response)

requests written
0 days 00:00:22.975677
requests written
0 days 00:00:08.142401
requests written
0 days 00:00:08.048316
requests written
0 days 00:00:08.021290
requests written
0 days 00:00:08.041309
****current_response ("The optimal strategy for stock speculators is to use a combination of fundamental and technical analysis to identify undervalued or overvalued stocks. The optimal instrument for speculation is a futures contract or a call option, as they offer leverage and the ability to profit from price movements without owning the underlying asset. However, it's essential to note that speculation carries significant risks, and investors should thoroughly assess their risk tolerance and financial situation before engaging in such activities. It's also crucial to diversify one's portfolio and set clear risk management strategies to mitigate potential losses. Ultimately, the optimal approach will depend on individual circumstances and market conditions. It is recommended to consult w

In [ ]:
opt.result